# 供电服务智能问答

In [35]:
import os
import json
import warnings
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, Audio
import sounddevice as sd
import soundfile as sf
import numpy as np

# 忽略警告
warnings.filterwarnings("ignore")

In [30]:
# 从同级目录的config.json读取配置
config_path = 'config.json'
with open(config_path) as f:
    config = json.load(f)

# 设置环境变量
for k, v in config.items(): os.environ[k] = v


client = OpenAI(api_key=config['OPENAI_API_KEY'], base_url=config['OPENAI_API_BASE'])

### 音频文件转文字

In [31]:
rudio_fp = "3224072400167607.mp3"

In [32]:
display(Audio(rudio_fp, autoplay=False))  

In [34]:
audio_file = open(rudio_fp, 'rb')  # 以二进制模式打开文件
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

# 默认返回值transcript为Transcription对象，访问方式如下
print(transcript.text)

通号11339为您服务 喂 你好 你好 我是东华街道同家村的 今天我们村在那里换那个电压器 然后我家里没电的 你现在家里没电是吧 我刚刚打到那个供电机里面 他说不是他们的问题 这我也不知道到底是什么问题了 那个电闸都是自己跳下来的 我推上去它又跳下来 你是 你打过我们所里电话了是吧 我打的是那个955多少来的 95598好像 你打的是95598 好像是打的那一个 在我们那个抢救人员跟你说 是表后的问题是不是 你是不是表后的问题 可能不是我们表情的问题 反正上面就是运行那个灯是亮的 就是我搬那个电闸 它就跳下来的 我搬上去它自己又跳下来 那些按钮都按过了也没用 那那个师傅跟你联系过了的是吧 那个师傅就是联系 他就说不是你们供电局的问题 说是我个人问题 这我又不知道去找谁 你的那个你可能是表后有故障 可能要那个找一下电工 找一下那个社会电工 电工我都不知道 社会电工你认识吗 我们同家村这边的是电工归水管 同家村 同家村是哪个 同家村是自然村 前面那个大村叫什么 前面一个大的自然村 就是死里铺在张网怀网这一块 死里铺 死里铺的存电工是谁 我也想不起来 死里铺 我们主要我们有那个产权分界线的 然后就是客户资产和我们资产 我们只处理供电公司资产 然后客户个人资产 我们才需要客户自己处理一下 我都不知道去找谁好 我跟无头苍蝇一样 还好遇到了你们的同事 我找他们要了个号码 那那个存电工 要不联系一下存电工 你们也有他的电话号码吗 我问一下吧 我看一下 要不我先打个电话 等一下我再回你电话好不好 好的好的 谢谢你啊 麻烦了 没事 再见


### 通话记录总结分析

In [ ]:
# 使用LangChain对通话记录进行结构化梳理
from IPython.display import Markdown
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.docstore.document import Document

# 初始化ChatOpenAI模型
llm = ChatOpenAI(
    model_name="gpt-4o-mini", 
    temperature=0,
    openai_api_key=config['OPENAI_API_KEY'],
    openai_api_base=config['OPENAI_API_BASE']
)

# 定义提示模板,要求模型提取关键信息
prompt_template = """
请对以下客服通话记录进行结构化梳理,提取以下要点:
1. 受理时间
2. 联系人
3. 联系地址
4. 受理内容
5. 后续跟进事项(如果有)

通话记录:
{text}

"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template
)

# 创建LLM链
chain = prompt | llm | StrOutputParser()

# 执行结构化分析
analysis = chain.invoke(transcript.text)
print("通话记录总结分析:")
Markdown(analysis)

通话记录总结分析:


以下是对客服通话记录的结构化梳理：

1. **受理时间**: 未明确提及
2. **联系人**: 东华街道同家村居民（未提供姓名）
3. **联系地址**: 东华街道同家村
4. **受理内容**: 
   - 家中停电，电闸跳闸，无法恢复供电。
   - 联系供电公司（95598），被告知问题可能不在供电公司。
   - 询问是否需要联系电工，因不清楚如何处理。
5. **后续跟进事项**: 
   - 客户需联系社会电工解决表后问题。
   - 客户表示会打电话联系电工，并会再回电给客服。

### 基于知识库进行答复 

In [ ]:
# 加载知识库文档
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 知识库路径
pdf_path = "./供电营业规则.pdf"

# 加载PDF文档
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# 文本分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(pages)

# 创建向量数据库
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)

# 创建检索器
retriever = vectorstore.as_retriever()

# 创建检索链
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)



C:\Users\19145\AppData\Local\Temp\ipykernel_7088\1285499334.py:23: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
# 定义问答函数
def get_answer(question):
    response = chain.run(question)
    print("问题:", question)
    print("\n回答:", response)
    return response

# 构建问题模板
question_template = """
请根据以下通话记录内容，结合供电营业规则知识库，给出专业的解决方案:

客户通话记录:
{text}

请分析客户问题并给出解决方案:
1. 明确客户的具体诉求是什么
2. 根据供电营业规则，提供相应的解决方案
3. 如果涉及具体流程，请列出办理步骤
4. 给出相关政策依据或条款说明

"""

# 创建问题模板
question_prompt = PromptTemplate(
    input_variables=["text"],
    template=question_template
)

# 创建问题链
question_chain = question_prompt | llm | StrOutputParser()

answer = question_chain.invoke({"text": transcript.text})
print("\n回答:")
Markdown(answer)



回答:


### 客户问题分析

1. **客户的具体诉求**：
   - 客户家中停电，且电闸频繁跳闸，无法恢复供电。
   - 客户希望了解停电原因，并寻求解决方案。
   - 客户对如何联系电工感到困惑，希望获得帮助。

### 解决方案

2. **根据供电营业规则，提供相应的解决方案**：
   - 根据通话记录，客户的电力问题可能是由于表后设备故障（如电线、插座、家用电器等）引起的，而供电公司只负责表前的供电设施。
   - 建议客户联系专业的电工进行检查和维修，确保电力设施的安全和正常运行。

3. **办理步骤**：
   - **确认问题**：客户需确认电闸跳闸的具体情况，是否有明显的短路或过载现象。
   - **联系电工**：客户可以通过以下方式联系电工：
     - 向邻居或村民询问推荐的电工。
     - 在当地社区或村委会询问是否有推荐的电工。
     - 如果有社交媒体或微信群，可以在群里询问。
   - **电工上门检查**：安排电工上门检查电路，确认故障原因。
   - **维修处理**：根据电工的检查结果，进行相应的维修或更换设备。

4. **相关政策依据或条款说明**：
   - 根据供电营业规则，供电公司负责供电设施的维护和管理，但客户家中的电力设施（如电线、电器等）属于客户个人资产，需由客户自行负责。
   - 供电公司在处理客户报修时，会根据故障的性质判断责任归属，若故障在表后，客户需自行联系专业电工进行处理。

### 总结
建议客户尽快联系专业电工进行检查和维修，以解决家中停电问题。同时，客户可以向邻居或村委会寻求帮助，找到合适的电工。供电公司将继续提供必要的支持和指导。